# GEOSPACIAL VISUALIZATION PROJECT

- Mi empresa:
    - Compañia de software con 50 empleados.
    - Producto: videojuegos
    - Empleados: 20 developers, 20 Designers/Creatieves/UX/UI and 10 executives/managers
     
- Requisitos buscados para localizar la empresa: radio de cercania: 50 km

        --> offices.latitude
        --> offices.longitude
        
    - Debe haber software engineers trabajando cerca:
        - category_code: 'software','web','biotech','mobile','hardware','games_video','ecommerce','enterprise','public_relations','consulting'
    
    - Debe haber un buen ratio entre grandes compañias / startups
        - number_of_employees: 0-100 small; 100-999 medium; >1000 big **
        - total_money_raised
        
    - Empresas cercanas deben cubrir los intereses de mis trabajadores --> Text mining
        - description
        - tag_list
        - twitter_username: nº de seguidores, nºtweets, text mining (topics)
        
    - Prefiero estar cerca de compañias nuevas que viejas
        - deadpooled_year: null -- ELimino las compañias que quebraron
        
        - founded_year -- considero solo compañias con 10 años de antiguedad maximo
      
    
    

CAMPOS DEL DATASET:

offices.latitude; offices.longitude; 
name, category_code
number_of_employees; total_money_raised
description; tag_list; twitter_username
founded_year; deadpooled_year: null

In [1]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

1. Crear un cursor con los campos que me interesan y filtrando de entrada ciertas empresas. 

In [2]:
companies_interest = db.companies.find({"deadpooled_year": None},
                                       {"_id":0,"name":1,"category_code":1,"number_of_employees":1,"ipo.valuation_amount":1,
                                       "total_money_raised":1,"description":1,
                                       "tag_list":1,"founded_year":1,"twitter_username":1,
                                       "offices.latitude":1,"offices.longitude":1})

companies_interest = list(companies_interest)

In [3]:
import pandas as pd

df_companies = pd.DataFrame(companies_interest)
df_companies.head(20)

df_companies.shape

(17872, 10)

In [4]:
df_companies.describe()

,founded_year,number_of_employees
count,12425.000000,8403.000000
mean,2002.243058,840.450315
std,13.328602,11384.038864
min,1800.000000,0.000000
25%,2001.000000,4.000000
50%,2006.000000,10.000000
75%,2008.000000,35.000000
max,2013.000000,405000.000000


In [5]:
#Change somethings in columns

df_= pd.DataFrame(pd.DataFrame(df_companies["offices"].values.flatten([0]).tolist()))
df_companies = pd.concat([df_,df_companies],axis=1)


#al aplicar flatten sobre la columna 'offices' se generan 62 columnas, la mayoria vacias. me quedo unicamente
#con la primera, que representa la primera oficina y donde la gran mayoria de las entradas presentan datos


/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
df_companies = df_companies[['name','category_code',0,'founded_year','number_of_employees',
                             'total_money_raised','description','tag_list','twitter_username']].copy()

In [7]:
df_companies.columns  = ['name','category','coords','founded_year','employees','total_money_raised',
                         'description','tags','twitter']

df_companies.head()

,name,category,coords,founded_year,employees,total_money_raised,description,tags,twitter
0,Digg,news,"{'latitude': 37.764726, 'longitude': -122.394523}",2004.0,60.0,$45M,user driven social content website,"community, social, news, bookmark, digg, techn...",digg
1,Flektor,games_video,"{'latitude': 34.025958, 'longitude': -118.379768}",NaN,NaN,$0,None,"flektor, photo, video",None
2,Postini,web,"{'latitude': 37.506885, 'longitude': -122.247573}",1999.0,NaN,$0,None,,None
3,Geni,web,"{'latitude': 34.090368, 'longitude': -118.393064}",2006.0,18.0,$16.5M,Geneology social network site,"geni, geneology, social, family, genealogy",geni
4,Fox Interactive Media,web,"{'latitude': 34.076179, 'longitude': -118.39417}",1979.0,0.0,$0,,,twitterapi


In [8]:
#elimino las filas que no tengan valor en founded year o en employees

import numpy as np

df_companies = df_companies[np.isfinite(df_companies['founded_year'])]
df_companies = df_companies[np.isfinite(df_companies['employees'])]

df_companies.isnull().sum()

name                     0
category                42
coords                1244
founded_year             0
employees                0
total_money_raised       0
description           2144
tags                   198
twitter               2651
dtype: int64

In [9]:
# cambio el tipo de dato ya que estaba como floats

df_companies['employees'] = df_companies['employees'].astype('int64')
df_companies['founded_year'] = df_companies['founded_year'].astype('int64')


In [10]:
# Creo bins con el número de empleados para diferenciar entre pequeña, mediana y gran empresa

bins = [0,10,100,999,500000]
labels = ["startup","small","medium","big"]
df_companies['tamaño_empresa'] = pd.cut(df_companies['employees'], bins=bins, labels=labels)

tamaño_count = df_companies.groupby(df_companies.tamaño_empresa)['name'].nunique()

tamaño_count

tamaño_empresa
startup    3906
small      2591
medium      674
big         211
Name: name, dtype: int64

In [11]:
# Extraer el numero en columna total money raised como indicador de tamaño de empresa

import re

def change_row (row):
    row = ''.join((ch if ch in '0123456789.' else ' ') for ch in row)
    return row

df_companies['total_money_raised'] = (df_companies['total_money_raised'].apply(change_row))

df_companies['total_money_raised(M)'] = df_companies['total_money_raised'].astype('float64')

df_companies = df_companies.reset_index()

df_companies = df_companies.drop(['total_money_raised','index'], axis=1,)

In [12]:
df_companies.head()

,name,category,coords,founded_year,employees,description,tags,twitter,tamaño_empresa,total_money_raised(M)
0,Digg,news,"{'latitude': 37.764726, 'longitude': -122.394523}",2004,60,user driven social content website,"community, social, news, bookmark, digg, techn...",digg,small,45.00
1,Geni,web,"{'latitude': 34.090368, 'longitude': -118.393064}",2006,18,Geneology social network site,"geni, geneology, social, family, genealogy",geni,small,16.50
2,Fox Interactive Media,web,"{'latitude': 34.076179, 'longitude': -118.39417}",1979,0,,,twitterapi,NaN,0.00
3,Scribd,news,"{'latitude': 37.789634, 'longitude': -122.404052}",2007,50,Read Unlimited Books,"book-subscription, digital-library, netflix-fo...",scribd,small,25.80
4,MeetMoi,social,"{'latitude': 40.757929, 'longitude': -73.985506}",2007,15,Mobile Dating,"mobile, dating, location, realtime, phone",meetmoi,small,5.58


In [13]:
df_companies.dropna(subset=['coords'], inplace=True)
df_companies.dropna(subset=['tamaño_empresa'], inplace=True)
df_companies.dropna(subset=['category'], inplace=True)


def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array['longitude'],array['latitude']]
    }


df_companies['coords'] = df_companies.apply(lambda r: toGeoJSON(r['coords']), axis=1)


df_companies.head()

,name,category,coords,founded_year,employees,description,tags,twitter,tamaño_empresa,total_money_raised(M)
0,Digg,news,"{'type': 'Point', 'coordinates': [-122.394523,...",2004,60,user driven social content website,"community, social, news, bookmark, digg, techn...",digg,small,45.00
1,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",2006,18,Geneology social network site,"geni, geneology, social, family, genealogy",geni,small,16.50
3,Scribd,news,"{'type': 'Point', 'coordinates': [-122.404052,...",2007,50,Read Unlimited Books,"book-subscription, digital-library, netflix-fo...",scribd,small,25.80
4,MeetMoi,social,"{'type': 'Point', 'coordinates': [-73.985506, ...",2007,15,Mobile Dating,"mobile, dating, location, realtime, phone",meetmoi,small,5.58
6,Plaxo,web,"{'type': 'Point', 'coordinates': [-122.055197,...",2002,50,Contact Management,"contact-management, address-book, sync, contac...",plaxo,small,28.30


In [14]:
# filtro categorias y año de fundacion (empresas que tengan maxixmo 10 años)

df_companies = df_companies.loc[df_companies['category'].isin(['news', 'web','games_video','mobile',
                                                               'photo_video', 'security', 'network_hosting',
                                                               'messaging', 'hardware', 'enterprise','software',
                                                               'ecommerce', 'public_relations', 'finance', 'music',
                                                               'analytics', 'biotech', 'sports','automotive',
                                                               'education','consulting', 'design', 'legal', 
                                                               'manufacturing'])]

df_companies = df_companies.loc[df_companies["founded_year"]>=2008]

df_companies = df_companies.reset_index()
df_companies = df_companies.drop(['index'], axis=1)
df_companies.head()

,name,category,coords,founded_year,employees,description,tags,twitter,tamaño_empresa,total_money_raised(M)
0,GoingOn,software,"{'type': 'Point', 'coordinates': [-122.392142,...",2008,40,Private Academic Networks,"socialnetworking, enterprise, social-media, so...",GoingOn_Network,small,8.5
1,Cloudant,enterprise,"{'type': 'Point', 'coordinates': [-71.05781, 4...",2008,70,NoSQL database managed service,"cloud-computing, big-data, databases, software...",cloudant,small,18.3
2,KIT digital,enterprise,"{'type': 'Point', 'coordinates': [-73.971655, ...",2008,200,video distribution solutions,,kitdigital,medium,20.0
3,YouBeQB,games_video,"{'type': 'Point', 'coordinates': [-122.151158,...",2008,8,,"youbeqb, fantasyfootball, fantasyleague, sports",,startup,0.0
4,Crunchyroll,games_video,"{'type': 'Point', 'coordinates': [-122.393229,...",2008,50,Digital Entertainment and Media Platform,"crunchyroll, anime, manga, video, gaming, musi...",crunchyroll,small,4.8


In [15]:
#elimino los registro que no tengan datos en coordenadas

df_companies = df_companies.loc[df_companies['coords']!= {'type': 'Point', 'coordinates': [None, None]}]

In [16]:
# guardar un nuevo json con las compañias seleccionadas y las coordenadas en formato geojson

df_companies.to_json("companies_clean.json", orient="records", lines=True)

In [17]:
df_companies.shape

(1043, 10)

In [18]:
df_companies['coords'][333]

{'coordinates': [-88.294854, 41.931912], 'type': 'Point'}

## Una vez el el json limpio y con las coordenadas en formato GeoJson, lo guardo y lo importo desde mongodb.

In [19]:
# Creo una lista con pares de coordenadas
list_names = list(df_companies['name'])
list_coord = list(df_companies['coords'])
list_coord = list (map(lambda d: d['coordinates'], list_coord))
list_names

dict_coord = dict(zip(list_names,list_coord))


In [20]:

def near_companies(coords): # esta función devuelve un dataframe con las empresas en un radio de 50km
    cursor = db.clean_companies.find({
      "coords": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [coords[0], coords[1]]
          },
          "$minDistance": 0,
          "$maxDistance":5000
        }
      }
    })
    
    df_geocomp = pd.DataFrame(list(cursor))
    
    return df_geocomp


def results (df_geocomp):    # no le he utilizado al final
    numero = df_geocomp['name'].count()
    category = df_geocomp['category'].value_counts()
    tamaño = df_geocomp['tamaño_empresa'].value_counts()
    return "Empresas:{}\nCategorias:\n{}\nTamaño:\n{}".format(str(numero),category,tamaño)


def results_df (df_geocomp): # esta tampoco aunque es la que me hubiera gustado usar
    results_df = df_geocomp[['name','category','tamaño_empresa']].copy()
    return results_df

def number (df_geocomp):
    return df_geocomp['name'].count()
    

In [40]:
# Empresas con el numero de empresas que hemos filtrado que tienen alrededor (64 es el max)

list_companies= []
for name, value in dict_coord.items():
    if int(number(near_companies(value)))>=64:
        list_companies.append(name)
print(list_companies)    

['Ninite', 'stickychicken', 'GitHub', 'Stitcher', 'SFentrepreneur', 'Bizroof', 'Posterous', 'EcoFlip', 'Advisor', 'Same Page Capital', 'Yammer', 'TopNotchThemes', 'Tivix', '148Apps', 'KODA', 'Startup Revival', 'LendingKarma', 'Fliptop', 'Finest Expert']


In [22]:
# Creo un diccionario nuevo solo con las empresas que han salido con un mayor de empresas cercanas elegidas (>62)

select_companies = {k: dict_coord[k] for k in dict_coord.keys() & {'GoingOn','Crunchyroll','Unknown Vector','YouNoodle','99designs','Ninite','stickychicken','GitHub',
        'Macroaxis','Serious Business','Stitcher','SFentrepreneur','Bizroof','Posterous','EcoFlip','OpenDrive',
        'Advisor','Allvoices','ngmoco','Same Page Capital','Twonq','Yammer','TopNotchThemes','Lolapps',
        'InnerRewards','Smule','AlphaClone','Tivix','Virgance','148Apps','Circle of Moms','Coveroo','5th Finger',
        'Meez','KODA','Startup Revival','Cloudize','The Vertical Action Group','Home-Account','Samasource',
        'Bamboudesign','LendingKarma','ArcTouch','AppNinjas','Fliptop','sipgate','Finest Expert','One Simple Move'}}

select_companies = pd.DataFrame.from_dict(select_companies, orient='index')

select_companies = select_companies.reset_index()

select_companies.columns = ['name','long','lat']

select_companies.head()

,name,long,lat
0,Crunchyroll,-122.393229,37.781265
1,Startup Revival,-122.419204,37.775196
2,Same Page Capital,-122.419415,37.774929
3,Samasource,-122.401803,37.778991
4,Macroaxis,-122.399878,37.789629


In [23]:
select_companies.to_json("companies_select.json",orient='records')

In [37]:
# filtro mas abajo para ver si sale alguna zona mas a parte de silicon valley
select_companies2 = {k: dict_coord[k] for k in dict_coord.keys() & {'GoingOn', 'KIT digital', 'Crunchyroll', 
                                                                    'Unknown Vector', 'YouNoodle', '99designs', 
                                                                    'Ninite', 'ReadBurner', 'stickychicken', 
                                                                    'Mojiva', 'GitHub', 'Salesconx', 'Sutros', 
                                                                    'Macroaxis', 'Serious Business', 'Fuzz', 
                                                                    'Stitcher', 'Knewton', 'SFentrepreneur', 
                                                                    'Pinch Media', 'Plentitube', 
                                                                    'Ladies Who Launch', 'Bizroof', 'Posterous', 
                                                                    'Open Places', 'EcoFlip', 'GigMaven', 
                                                                    'Scavenja', 'OpenDrive', 'Advisor', 'Allvoices',
                                                                    'ngmoco', 'Same Page Capital', 'YellowSpaces',
                                                                    'plaYce', 'Thomson Reuters', 'Twonq', 
                                                                    'Papervitamins', 'Yammer', 'TopNotchThemes', 
                                                                    'Lolapps', 'Samepoint', 'InnerRewards', 'Unigo',
                                                                    'Smule', 'Sailthru', 'Buzzr', 'AlphaClone', 
                                                                    'RGA Advisors, LLC', 'Yipit', 'Tivix', 'Virgance',
                                                                    '148Apps', 'People Capital', 'CyberSynchs', 
                                                                    'Circle of Moms', 'Coveroo', 'Indee', '5th Finger', 
                                                                    'Meez', 'KODA', 'Music For Busy People', 'SwitchGames',
                                                                    'Startup Revival', 'Cloudize', 'Patch', 
                                                                    'The Vertical Action Group', 'Wee Web', 
                                                                    'IrishCentral', 'Kapitall', 'Home-Account',
                                                                    'IntheGlo', 'Chartbeat', 'RipCloud', 'Samasource',
                                                                    'TrialX', 'Bamboudesign', 'LendingKarma', 'ArcTouch', 
                                                                    'AppNinjas', 'Fliptop', 'ArtNeighbor', 'sipgate',
                                                                    'Covet Shop', 'Udorse', 'Finest Expert', 'One Simple Move',
                                                                    'Strategist News', 'ChallengePost', 'StyleCaster', 'Conversition'}}
                     
select_companies2 = pd.DataFrame.from_dict(select_companies2, orient='index')

select_companies2 = select_companies2.reset_index()

select_companies2.columns = ['name','long','lat']
                     
select_companies2.to_json("companies_select2.json",orient='records')

In [41]:
# solo las que tienen 64 cercanas, el maximo
select_companies3 = {k: dict_coord[k] for k in dict_coord.keys() & {'Ninite', 'stickychicken', 'GitHub', 'Stitcher',
                                                                    'SFentrepreneur', 'Bizroof', 'Posterous', 'EcoFlip',
                                                                    'Advisor', 'Same Page Capital', 'Yammer',
                                                                    'TopNotchThemes', 'Tivix', '148Apps', 'KODA',
                                                                    'Startup Revival', 'LendingKarma', 'Fliptop',
                                                                    'Finest Expert'}}
                     
select_companies3 = pd.DataFrame.from_dict(select_companies3, orient='index')

select_companies3 = select_companies3.reset_index()

select_companies3.columns = ['name','long','lat']
                     
select_companies3.to_json("companies_select3.json",orient='records')

In [24]:
companies = df_companies[['name','category','coords','tamaño_empresa']]

df_= pd.DataFrame(pd.DataFrame(companies["coords"].values.flatten([0]).tolist()))
companies = pd.concat([df_,companies],axis=1)


coord = companies['coordinates'].apply(pd.Series)
companies = pd.concat([companies[:], coord[:]], axis=1)
companies = companies.drop(['coords','type','coordinates'], axis=1)

companies.columns = ['name','category','tamaño_empresa','long','lat']

companies.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,category,tamaño_empresa,long,lat
0,GoingOn,software,small,-122.392142,37.782263
1,Cloudant,enterprise,small,-71.057810,42.358920
2,KIT digital,enterprise,medium,-73.971655,40.751816
3,YouBeQB,games_video,startup,-122.151158,47.597965
4,Crunchyroll,games_video,small,-122.393229,37.781265


In [25]:
companies = companies.dropna()

In [26]:
companies = companies.loc[companies['category'].isin(['web','games_video','mobile','network_hosting','hardware',
                                                      'software','ecommerce','public_relations','finance',
                                                      'analytics','sports','design'])]

In [27]:
companies.to_json("companies_all.json",orient='records')